### Packages

In [1]:
from PIL import Image
import cv2
import numpy as np

import matplotlib.pyplot as plt

### Gradient

In [2]:

#* Read Image in GrayScale
img_gray = cv2.imread('images/lena.jpg',0) 
h, w = img_gray.shape[:2]

grad_img = np.asarray(img_gray)

In [3]:
for i in range(0, h):
    for j in range(0, w - 1):
        a = min(img_gray[i][j + 1], img_gray[i][j])
        if a == img_gray[i][j + 1]:
            temp_arr = img_gray[i][j] - img_gray[i][j + 1]
        else:
            temp_arr = img_gray[i][j + 1] - img_gray[i][j]
            grad_img[i, j] = temp_arr

In [4]:
img = Image.fromarray(grad_img)                              
img.save("images/gradient.jpg")

### Image Negative

In [5]:
S = 255
rgb_path = 'images/rgb.jpg'
binary_path = 'images/binary.jpg'
gray_path = 'images/grayscale.png'

#### RGB Image

In [6]:
img = cv2.imread(rgb_path, cv2.IMREAD_COLOR)
B, G, R = cv2.split(img)

In [7]:
B[:] = [S - x for x in B]     #? Inverting Blue Color
G[:] = [S - x for x in G]     #* Inverting Green Color    
R[:] = [S - x for x in R]     #! Inverting Red Color

my_img = cv2.merge((B, G, R)) 
cv2.imwrite('images/rgb_inverted.png', my_img)

True

#### Binary and Grayscale Image

In [8]:
img = cv2.imread(binary_path ,cv2.IMREAD_GRAYSCALE) 
my_img = np.array([S-x for x in img])
cv2.imwrite('images/binary_inverted.png', my_img)

True

In [9]:
img = cv2.imread(gray_path ,cv2.IMREAD_GRAYSCALE) 
my_img = np.array([S-x for x in img])
cv2.imwrite('images/gray_inverted.png', my_img)

True

### Segmentation

In [10]:

#* Loading the image in RGB
img = cv2.imread("images/seg_image.png", 0)

In [11]:

#* Applying Gaussian blur with kernel size 7 to remove unwanted noise
#* Usually done during downsampling to remove high frequency noise
blurred_image = cv2.GaussianBlur(img, (7, 7), 0)

In [12]:

#* Applying Otsu's thresholding to binarize the image
retval, binarized_image = cv2.threshold(blurred_image, 40, 255, cv2.THRESH_BINARY)

In [13]:

#* Applying Closing to fill in the holes
filter = np.ones((3, 3), np.uint8)
closed_image = cv2.morphologyEx(binarized_image, cv2.MORPH_CLOSE, filter)

In [14]:

#* Using connected components to label the image
retval, markers = cv2.connectedComponents(closed_image)

In [15]:

#* Mapping the component labels to hue val
label_hue = np.uint8(120*markers/np.max(markers))
blank_ch = 255 * np.ones_like(label_hue)
labeled_image = cv2.merge([label_hue, blank_ch, blank_ch])

In [25]:

#* changing from HSV to RGB again to show
labeled_image = cv2.cvtColor(labeled_image, cv2.COLOR_HSV2BGR)

In [30]:

#* background label set to black
labeled_image[label_hue == 0] = 0

In [31]:

#* getting the unique colors in the image
unique_colors = np.unique(labeled_image.reshape(-1, labeled_image.shape[2]), axis = 0)

In [33]:
print ("Colors available in labeled image:")
for x in range(unique_colors.shape[0]):
    print (str(x+1)+"=> B:"+str(unique_colors[x,0])+"    G:"+str(unique_colors[x,1])+"   R:"+str(unique_colors[x,2])+" ")

Colors available in labeled image:
1=> B:0    G:0   R:0 
2=> B:0    G:255   R:0 
3=> B:255    G:0   R:0 


In [34]:
r = 0
b = 255
g = 255

In [35]:

#* making an output image
output_image = np.zeros_like(labeled_image)

#* getting the object of user input color
for x in range(labeled_image.shape[0]):
    for y in range(labeled_image.shape[1]):
        if (labeled_image[x,y,0] == int(r) and labeled_image[x,y,1] == int(g) and labeled_image[x,y,2] == int(b)):
            output_image[x,y,0:3] = labeled_image[x,y,0:3]

In [36]:
cv2.imwrite('images/segmentated.png', labeled_image)

True

### Centroid

In [37]:
img = cv2.imread("images/Signature.png", 0)
ret,binary = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY)

In [39]:
def findBB(im):
    h, w = im.shape[0], im.shape[1] 
    left, top = w, h
    right, bottom = 0, 0
    
    for x in range(h):
        for y in range(w):
            if (im[x,y] == 0):
                right = x if x > right else right
                left = x if x < left else left
                bottom = y if y > bottom else bottom
                top = y if y < top else top
                
    return (left, right, top, bottom)

In [41]:
def findCentroid(im):
    h, w = im.shape[0], im.shape[1]
    cx, cy, n = 0, 0, 0
    for x in range(h):
        for y in range(w):
            if (im[x,y] == 0):
                cx += x
                cy += y
                n += 1
    cx /= n
    cy /= n
    return (cx, cy)

In [42]:
def divideImgIntoFour(im, cent):
    h, w = im.shape[0], im.shape[1]
    cx, cy = cent
    cx, cy = int(cx), int(cy)
    img1 = im[0:cx, 0:cy]
    img2 = im[0:cx, cy:w]
    img3 = im[cx:h, 0:cy]
    img4 = im[cx:h, cy:w]
    return [img1, img2, img3, img4]

In [45]:
def calculateTransitions(im):
    h, w = im.shape[0], im.shape[1]
    prev = im[0,0]
    n = 0
    for x in range(1, h):
        for y in range(1, w):
            curr = im[x,y]
            # check if the is black to white transition
            n = n+1 if curr == 255 and prev == 0 else n
            prev = curr
    return n

In [47]:
boundingBox = findBB(binary)
cropImg = binary[boundingBox[0]:boundingBox[1], boundingBox[2]:boundingBox[3]]
centroid = findCentroid(cropImg)
segments = divideImgIntoFour(cropImg, centroid)
# transitions = [calculateTransitions(seg) for seg in segments]

print ("Bounding Box:", boundingBox)
print ("Coordinates of centroid:", centroid)
# print ("Black to white transitions (4 segments):", transitions)

Bounding Box: (133, 440, 11, 566)
Coordinates of centroid: (129.08519840309702, 299.19722356641665)


In [48]:
# cv2.imshow("TopLeft", segments[0])
cv2.imwrite("images/TopLeft.png", segments[0])
# cv2.imshow("TopRight", segments[1])
cv2.imwrite("images/TopRight.png", segments[1])
# cv2.imshow("BottomLeft", segments[2])
cv2.imwrite("images/BottomLeft.png", segments[2])
# cv2.imshow("BottomRight", segments[3])
cv2.imwrite("images/BottomRight.png", segments[3])

True